# Where was I

I use the following notebook to count how many days I spent on which location, leveraging [Google Maps Timeline](https://support.google.com/maps/answer/6258979?hl=en&co=GENIE.Platform%3DDesktop).

## How to use

1. Install geopandas, shapely, pandas

```terminal
pip install pandas, shapely, geopandas
```

2. Download your location history data from [Google Takeout](https://takeout.google.com/)

3. Download a shape file with country borders. For example I use the [1:50 cultural vectors from Natural Earth](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/)

4. Unzip the downloads, and note the path to the relevant files in the `DATA_PATH` and `SHAPE_FILE` variables

5. Run the cells. You may wish to adjust the date range in the final cell.

In [1]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point

In [2]:
DATA_PATH = "Takeout/Location History/Records.json"
SHAPE_FILE = "ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp"

In [3]:
gdf = gpd.read_file(SHAPE_FILE)

In [4]:
# reading/parsing location history data
with open(DATA_PATH) as f:
    data = json.load(f)
data = data['locations']

In [5]:
# convert to pandas
data_df = pd.DataFrame(data)
del data

In [6]:
# dont care about other cols
data_df = data_df[['latitudeE7', "longitudeE7","timestamp"]]

In [7]:
# appropriately format lat, lon
data_df["latitude"] = data_df["latitudeE7"] / 1e7
data_df["longitude"] = data_df["longitudeE7"] / 1e7

In [8]:
# convert lat lon into geopandas geometry obj
geometry = [
    Point(lon, lat) for lat, lon in zip(data_df["latitude"], data_df["longitude"])
]
data_df = gpd.GeoDataFrame(data_df, geometry=geometry)
data_df.crs = 'EPSG:4326'

In [ ]:
# perform spatial join (takes a bit)
data_df = gpd.sjoin(data_df, gdf, how='left', predicate='within')

In [ ]:
# convert to proper type
data_df["timestamp"] = pd.to_datetime(data_df["timestamp"], format='mixed')

In [ ]:
# index by the timestamp column
data_df = data_df.set_index("timestamp")

In [ ]:
# only care about these columns
data_df = data_df[['latitude', 'longitude', 'NAME']]

In [ ]:
# better column name
data_df = data_df.rename(columns={"NAME": "country"})

In [ ]:
# change the date range accordingly
data_df["2023/01/01":"2023/12/31"].resample('D').ffill().groupby('country').size()